### Import libraries

In [1]:
import numpy as np 
import scipy.sparse as ss
from scipy.linalg import norm
import qiskit
from qiskit.quantum_info import Statevector,DensityMatrix,partial_trace
from qiskit.quantum_info import entanglement_of_formation, entropy
import h5py

In [2]:
# Define the number of qubit
global qubit 
qubit = 4

In [3]:
# function to define labels
def labelCreation(qubit:int):
    labels = []
    for j in range(2**qubit):
        b = bin(j).replace("0b","")
        if len(b) < qubit:
            n = qubit - len(b)
            b = n*"0"+b
        labels.append(b)
    return labels

In [4]:
# function to define statevector and density matrix 
def StateCreation(coef,labels):
    if np.isclose(np.linalg.norm(coef),1):
        ψ = Statevector(np.zeros(2**qubit))
        for j in range(len(labels)):
            ψ += coef[j]*Statevector.from_label(labels[j])
        ρ = DensityMatrix(ψ)
    else:
        raise TypeError("The coefficient vector is not normalized")
    return ψ, ρ

In [5]:
# check created labels 
labels = labelCreation(qubit)

In [6]:
# Coefficients determination 
coef = np.zeros(2**qubit)
coef[0] = 0.5;
coef[5] = 0.5;
coef[10] = 0.5;
coef[15] = 0.5;

In [7]:
# Check labels with their coefficients

for j in range(len(labels)):
    print("j = {}, coef = {}, label = {}".format(j,coef[j],labels[j]))

j = 0, coef = 0.5, label = 0000
j = 1, coef = 0.0, label = 0001
j = 2, coef = 0.0, label = 0010
j = 3, coef = 0.0, label = 0011
j = 4, coef = 0.0, label = 0100
j = 5, coef = 0.5, label = 0101
j = 6, coef = 0.0, label = 0110
j = 7, coef = 0.0, label = 0111
j = 8, coef = 0.0, label = 1000
j = 9, coef = 0.0, label = 1001
j = 10, coef = 0.5, label = 1010
j = 11, coef = 0.0, label = 1011
j = 12, coef = 0.0, label = 1100
j = 13, coef = 0.0, label = 1101
j = 14, coef = 0.0, label = 1110
j = 15, coef = 0.5, label = 1111


In [9]:
# Create quantum state 
ψ, ρ = StateCreation(coef, labels)

In [9]:
# take a look at created state 
ψ.draw('latex')

<IPython.core.display.Latex object>

In [10]:
# take a look at created density matrix
ρ.draw('latex')

<IPython.core.display.Latex object>

In [7]:
# Define a function to take  partial trace 
# This func takes two input, density matrix and a list 
# In the input list, we are given the number of part we should partial trace over 

def PartialTrace(ρ,L:list):
    """
    ρ: Density matrix
    List: this list just can take 1,2,3
    """
    pt1 = []
    pt2 = []
    pt3 = []
        
    if np.isin(1,L) and len(ρ.dims()) > 1:
        for j in range(len(ρ.dims())):
            pt1.append(partial_trace(ρ,[j]))
    
    if np.isin(2,L) and len(ρ.dims()) > 2:
        for j in range(len(ρ.dims())-1):
            pt2.append(partial_trace(ρ,[j,j+1]))
            if j+2 < len(ρ.dims()):
                pt2.append(partial_trace(ρ,[j,j+2]))
                if j+3 < len(ρ.dims()):
                    pt2.append(partial_trace(ρ,[j,j+3]))
                    
    if np.isin(3,L) and len(ρ.dims()) > 3:
        for j in range(len(ρ.dims())-2):
            pt3.append(qiskit.quantum_info.partial_trace(ρ,[j,j+1,j+2]))
            if j+3 < len(ρ.dims()):
                pt3.append(qiskit.quantum_info.partial_trace(ρ,[j,j+1,j+3]))
                if j+1 < len(ρ.dims())-2:
                    pt3.append(qiskit.quantum_info.partial_trace(ρ,[j,j+2,j+3]))
                    
    partial_trace_func = {
        "partial_trace_1" : pt1,
        "partial_trace_2" : pt2,
        "partial_trace_3" : pt3
    }
                        
    return partial_trace_func

In [10]:
# Take partial trace over one part 
ρ_pt_1 = PartialTrace(ρ, [1])
len(ρ_pt_1['partial_trace_1'])

4

In [11]:
# Function to compute entanglement of formation(EOF) for a three-partite system
def EOF_3(ρ_pt:dict): 
    """
    ρ_pt : A dictionary containing partial trace over density matrix 
    """
    eof_1 = []
    eof_2 = []
    entropy_1 = []
    entropy_2 = []
    if len(ρ_pt['partial_trace_1']) == 0:
        pass
    else:
        for j in range(len(ρ_pt['partial_trace_1'])):
            eof_1.append(entanglement_of_formation(
                ρ_pt['partial_trace_1'][j]))
            entropy_1.append(entropy(ρ_pt['partial_trace_1'][j]))
    
    if len(ρ_pt['partial_trace_2']) == 0:
        pass 
    else:
        for j in range(len(ρ_pt['partial_trace_2'])):
            eof_2.append(entanglement_of_formation(
                ρ_pt['partial_trace_1'][j]))
            entropy_2.append(entropy(ρ_pt['partial_trace_2'][j]))
            
            
    eof_1 = np.array(eof_1)
    eof_2 = np.array(eof_2)
    entropy_1 = np.array(entropy_1)
    entropy_2 = np.array(entropy_2)
    EOF = np.sum(eof_1+eof_2+entropy_1+entropy_2)/6
    return EOF

In [12]:
V = np.zeros((8,1))
D = []
start = -1.0;
stop = 1.0;
num = 5;

for v0 in np.linspace(start, stop, num):
    V[0,0] = v0
    for v1 in np.linspace(start, stop, num):
        V[1,0] = v1
        for v2 in np.linspace(start, stop, num):
            V[2,0] = v2
            for v3 in np.linspace(start, stop, num):
                V[3,0] = v3
                for v4 in np.linspace(start, stop, num):
                    V[4,0] = v4
                    for v5 in np.linspace(start, stop, num):
                        V[5,0] = v5
                        for v6 in np.linspace(start, stop, num):
                            V[6,0] = v6
                            for v7 in np.linspace(start, stop, num):
                                V[7,0] = v7
                                ρ = DensityMatrix(np.outer(V,V))
                                ρ_2 = DensityMatrix(np.matmul(ρ.data,ρ.data))
                                if ρ.is_valid() and ρ_2.is_valid():
                                    D.append(ρ)
                                    
print(len(D))

1136


In [132]:
D2 = D[0:100]
len(D2)

100

In [56]:
# My ans 
ans = []
for j in range(3):
    M = ρ_pt_1["partial_trace_1"][j].data
    for k in range(len(D2)):
        A = D2[k].data
        for p in np.linspace(0,1,100):
            for kk in range(len(D)):
                B = D[kk].data

1212


In [84]:
M = ρ_pt_1["partial_trace_1"][2].data
# M == p*A + (1-p)*B
np.any(np.isclose(M,(p*A + (1-p)*B)))

True

In [85]:
np.isclose(M,(p*A + (1-p)*B))

array([[ True, False, False, False,  True,  True, False,  True],
       [False,  True, False, False,  True,  True,  True, False],
       [False, False, False, False,  True,  True,  True,  True],
       [False, False, False, False,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True],
       [False,  True,  True,  True,  True,  True, False,  True],
       [ True, False,  True,  True,  True,  True,  True, False]])

In [46]:
# Javidan ans
ans = []
for j in range(3):
    M = ρ_pt_1["partial_trace_1"][j].data
    for k in range(len(D2)):
        A = D2[k].data
        for kk in range(len(D2)):
            B = D2[kk].data
            if not A[0,0] - B[0,0] == 0:
                p = (M[0,0] - B[0,0])/(A[0,0] - B[0,0])
                if norm(M - (p*A+(1-p)*B))<1e-3:
                    ans.append((p,A,B))

print(len(ans))

0


In [55]:
# Javidan ans
ans = []
for j in range(3):
    M = ρ_pt_1["partial_trace_1"][j].data
    for k in range(len(D2)):
        A = D2[k].data
        for kk in range(len(D2)):
            B = D2[kk].data
            for a in range(8):
                for b in range(8):
#                     if not A[a,b] - B[a,b] == 0:
                        
# #                         print(A[a,b], B[a,b])
#                         p = (M[a,b] - B[a,b])/(A[a,b] - B[a,b])
    
# print(p)                       
#         print(p)
#                         if 0 < np.real(p) <= 1:
#                             print(np.real(p))
#                         break
                        
#             if norm(M - (p*A+(1-p)*B))<1e-3:
#                 ans.append((p,A,B))

# print(len(ans))

In [83]:
M = ρ_pt_1["partial_trace_1"][1].data

for j in range(len(D2)):
    A = D2[j].data
    for k in range(len(D2)):
        B = D2[k].data
#         p = 0
        if (A-B).all() == 0: 
#             print("k")
            pass 
        for a in range(8):
            for b in range(8):
                if not A[a,b] == B[a,b] :
                    p = (M[a,b] - B[a,b]) / (A[a,b] - B[a,b])
                    
                break
            break
        print(p)
#                 if 0 < p < 1:
#                     print(p)
                

(1-0j)
0j
0j
0j
0j
(1-0j)
(1-0j)
(1-0j)
(1-0j)
(1-0j)
(1-0j)
(1-0j)
(1-0j)
(1-0j)
(1-0j)
(1-0j)
(1-0j)
(1-0j)
(1-0j)
(1-0j)
(1-0j)
(1-0j)
(1-0j)
(1-0j)
(1-0j)


In [133]:
M =ρ_pt_1["partial_trace_1"][0]
s = 0
ans = []
for j in range(len(D2)):
    for k in range(len(D2)):
        for p1 in np.arange(0,1,100):
            for p2 in np.arange(0,1,100):
                if M == p1*D2[j] + p2*D2[k]:
                    s += 1
                    
print(s)

0


In [ ]:
hf = h5py.File("Dinkhah_data.h5","w")
hf.create_dataset("ans", data=ans)
hf.close()